## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [0]:
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir('/content/drive/My Drive/Neural Networks/R6')
os.listdir()

['11_Iris.csv',
 'R6_Internal_Lab_UpdatedTF2_Prices_Iris-3.ipynb',
 'prices.csv',
 'NeuralNetworks_Prices_IRIS.h5']

In [0]:
import pandas as pd
import numpy as np

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
path_files = '/content/drive/My Drive/Neural Networks/R6/'

In [0]:
data = pd.read_csv(path_files + 'prices.csv')

### Check all columns in the dataset

In [9]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis = 1, inplace = True)

In [11]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_first_1000 = data.loc[:999,:]

In [13]:
data_first_1000.shape

(1000, 5)

In [14]:
data_first_1000['volume'] = data_first_1000['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
data_first_1000.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data_first_1000.drop('volume', axis = 1)

In [0]:
y = data_first_1000['volume']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 5)

In [20]:
X_train.shape

(700, 4)

In [21]:
y_train.shape

(700,)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [25]:
X_train

array([[0.5020444 , 0.5008865 , 0.49440256, 0.5026233 ],
       [0.50310385, 0.4968752 , 0.49663866, 0.5033404 ],
       [0.502044  , 0.5017134 , 0.49196017, 0.50419307],
       ...,
       [0.49686557, 0.501396  , 0.49621835, 0.5054641 ],
       [0.50167567, 0.4976353 , 0.49696192, 0.50369585],
       [0.50258434, 0.49776903, 0.49703524, 0.50258434]], dtype=float32)

In [0]:
X_test = transformer.transform(X_test)

In [27]:
X_test

array([[0.49682367, 0.5021781 , 0.49633688, 0.50461185],
       [0.49373037, 0.5076786 , 0.4902433 , 0.5080888 ],
       [0.5011712 , 0.49767655, 0.4975626 , 0.5035642 ],
       ...,
       [0.50243956, 0.500157  , 0.49492624, 0.50243956],
       [0.501209  , 0.4976006 , 0.49687892, 0.50427616],
       [0.49718228, 0.5036043 , 0.49488866, 0.5042596 ]], dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [32]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)))

Current Loss on iteration 0 tf.Tensor(255.41042, shape=(), dtype=float32)
Current Loss on iteration 1 tf.Tensor(253.51328, shape=(), dtype=float32)
Current Loss on iteration 2 tf.Tensor(251.7649, shape=(), dtype=float32)
Current Loss on iteration 3 tf.Tensor(250.15358, shape=(), dtype=float32)
Current Loss on iteration 4 tf.Tensor(248.66856, shape=(), dtype=float32)
Current Loss on iteration 5 tf.Tensor(247.30002, shape=(), dtype=float32)
Current Loss on iteration 6 tf.Tensor(246.03874, shape=(), dtype=float32)
Current Loss on iteration 7 tf.Tensor(244.87634, shape=(), dtype=float32)
Current Loss on iteration 8 tf.Tensor(243.80505, shape=(), dtype=float32)
Current Loss on iteration 9 tf.Tensor(242.81778, shape=(), dtype=float32)
Current Loss on iteration 10 tf.Tensor(241.90788, shape=(), dtype=float32)
Current Loss on iteration 11 tf.Tensor(241.06932, shape=(), dtype=float32)
Current Loss on iteration 12 tf.Tensor(240.29652, shape=(), dtype=float32)
Current Loss on iteration 13 tf.Tens

### Get the shapes and values of W and b

In [33]:
print(w.shape)

(4, 1)


In [34]:
print(b.shape)

(1,)


In [35]:
w.numpy()

array([[1.2575908],
       [1.2621496],
       [1.2447186],
       [1.273089 ]], dtype=float32)

In [36]:
b.numpy()

array([2.51892], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [0]:
test_prediction = prediction(X_test[0:1], w, b)

In [38]:
print(test_prediction.numpy())

[[5.03776]]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_data = pd.read_csv(path_files + '11_Iris.csv')

In [40]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le = LabelEncoder()

iris_data['Species'] = le.fit_transform(iris_data['Species'])

In [43]:
iris_data['Species'].unique()

array([0, 1, 2])

In [0]:
iris_data = pd.get_dummies(data = iris_data, columns=['Species'])

In [45]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_0,Species_1,Species_2
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris_data.drop(['Species_0','Species_1', 'Species_2','Id'], axis = 1)

y = iris_data[['Species_0','Species_1', 'Species_2']]

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 5)

### Model Training 

In [49]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=100,
          batch_size=X_train.shape[0])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 723us/sample - loss: 3.1314 - acc: 0.3238 - val_loss: 2.6450 - val_acc: 0.3556
Epoch 2/100
105/105 [==============================] - 0s 55us/sample - loss: 2.7526 - acc: 0.3238 - val_loss: 2.3168 - val_acc: 0.3556
Epoch 3/100
105/105 [==============================] - 0s 49us/sample - loss: 2.4019 - acc: 0.3238 - val_loss: 2.0312 - val_acc: 0.3556
Epoch 4/100
105/105 [==============================] - 0s 50us/sample - loss: 2.0952 - acc: 0.3238 - val_loss: 1.8018 - val_acc: 0.3556
Epoch 5/100
105/105 [==============================] - 0s 48us/sample - loss: 1.8467 - acc: 0.3238 - val_loss: 1.6313 - val_acc: 0.3778
Epoch 6/100
105/105 [==============================] - 0s 55us/sample - loss: 1.6604 - acc: 0.3238 - val_loss: 1.5092 - val_acc: 0.4444
Epoch 7/100
105/105 [=======================

In [50]:
model.get_weights()

[array([[-0.3581699 , -0.18571015,  0.07758212],
        [ 0.9849658 ,  0.21411744, -0.37168783],
        [-0.13432586,  0.6393134 ,  0.17132826],
        [ 0.22336312, -0.5739029 ,  0.99934065]], dtype=float32),
 array([ 0.10058332, -0.06767137, -0.03291184], dtype=float32)]

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


### Model Prediction

In [52]:
model.predict(X_test[0:1])

array([[0.17857409, 0.4018982 , 0.41952774]], dtype=float32)

### Save the Model

In [0]:
model.save('NeuralNetworks_Prices_IRIS.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?